In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from skimage.feature import peak_local_max
import mrcfile
import napari
import starfile

In [26]:
tomogram_path = Path("/mnt/scratch/ribosomes/kas_k44a/isonet/corrected_iter30/ts_003_b10_rec_corrected.mrc").absolute()
segmentation_path = Path("/mnt/scratch/ribosomes/kas_k44a/eman2/segmentations/ts_003_b10_rec_corrected__good_ribosomes-6.922_seg.mrc").absolute()

ts_name = "ts_003"

In [ ]:
tomogram_mrc = mrcfile.read(tomogram_path)
segmentation_mrc = mrcfile.read(segmentation_path)

In [16]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(
    tomogram_mrc,
    name=ts_name,
    depiction="plane",
    blending="translucent",
    visible=False,
)
viewer.add_image(
    segmentation_mrc, 
    name="segmentation",
    depiction="volume",
    blending="translucent",
    colormap="gist_earth",
    visible=False,
)

In [40]:
tomo_bin_factor = 10
min_peak_distance_px = 10
threshold_abs = 4
threshold_rel = None
border_size_in_px = 5

rln_coordinates = ["rlnCoordinateZ", "rlnCoordinateY", "rlnCoordinateX"]
rln_angles = ["rlnAngleRot", "rlnAngleTilt", "rlnAnglePsi"]

# Load the segmentation data, identify particle positions, and write them to a star file
peaks = peak_local_max(
    segmentation_mrc,
    min_distance=min_peak_distance_px,
    threshold_abs=threshold_abs,
    threshold_rel=threshold_rel,
    exclude_border=border_size_in_px,
)

# Convert the peak coordinates to RELION's coordinate system
rln_df = pd.DataFrame(peaks * tomo_bin_factor, columns=rln_coordinates)

# Add random orientations to each peak
for angle in rln_angles:
    random_angles = np.random.randint(0, 179, size=len(rln_df.index))
    rln_df[angle] = random_angles

rln_df["rlnTomoName"] = ts_name

# Write the star file
starfile_name = f"peaks_abs{threshold_abs}_dist{min_peak_distance_px}_{ts_name}.star"
peaks_directory = Path("/mnt/scratch/ribosomes/kas_k44a/eman2/").absolute()
starfile.write(rln_df, peaks_directory / starfile_name, overwrite=True)




In [41]:
viewer.add_points(
    peaks,
    name=f"peaks_dist{min_peak_distance_px}_abs{threshold_abs}",
    blending="translucent",
    edge_color="red",
    edge_width=0.2,
    visible=False,
)


<Points layer 'peaks_dist10_abs4' at 0x7f2a87ba0be0>